# 6.1 パラメータチューニング

## 6.1.1 ハイパーパラメータの探索手法

1. 手動
2. グリッドサーチ/ランダムサーチ  
   パラメータ空間を予め指定した範囲を規則的に探索/予め指定した手法でランダムに探索  
   `sckikit-learn.model_selection` の `GridSearchCV`, `RandomizedSearchCV` など
3. ベイズ最適化 (Bayseian Optimization)  
    以前に計算したパラメータの履歴に基づいてベイズの手法を用いて選択
    `hyperopt`, `optuna` など

経験的には、kaggle でのパラメータ探索は手で行っている人が多い印象。

## 6.1.2 パラメータチューニングで設定すること

1. ベースラインとなるパラメータ
2. 探索する対象となるパラメータとその範囲
3. 手動で調節するか、自動的に探索するか
4. 評価の枠組み (fold の分け方など)

パラメータを自動で調節する場合には、

- パラメータチューニングをしすぎて学習データに過剰に適合してしまう
- 計算時間が長くなる

といった問題が起こり得るので、

- チューニングとモデルの作成をする際の fold の分け方を変える
- validation の fold の 1 つだけを用いて精度を確認する

などの対策をしたほうが良い。

## 6.1.3 パラメータチューニングのポイント

各モデルには結果を大きく左右する大事なパラメータが存在する。  
そのため、パラメータを探索する際はその**モデルの重要なパラメータから調節していく**ことが大事である。  

また、**モデルとパラメータの関係を理解して得られた結果からなぜそうなったのかを理解すること**で、次にどうパラメータを変化させるべきかを考えながらチューニングしていくとよい。

なお、GBDT ではパラメータチューニングよりも良い特徴量を追加するほうが有用なことが多いので、あまりチューニングに時間を割かない方が良い。


## 6.1.4 ベイズ最適化でのパラメータ探索

### `hyperopt`




### `optuna`

## 6.1.5 ベイズ最適化でのパラメータ探索